### Q1 A Download the Anuran Calls (MFCCs) Data Set from: https://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29. Choose 70% of the data randomly as the training set.


In [20]:
import pandas as pd 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import hamming_loss
import warnings
from sklearn import preprocessing

In [21]:
data= pd.read_csv('Frogs_MFCCs.csv')

In [22]:
data_y=data[['Family','Genus','Species']]
data_x=data.drop(['Family','Genus','Species','RecordID'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(data_x,data_y, test_size=0.30, random_state=42)




## Q1 B 1 Research exact match and hamming score/ loss methods for evaluating multi-label classication and use them in evaluating the classifiers in this problem.

### classification and use them in evaluating the classifiers in this problem.
Exact match : indicates the percentage of samples that have all their labels classified correctly,   a chance of being partially correct, but here we ignore those partially correct matches

Hamming loss : Hamming-Loss is the fraction of labels that are incorrectly predicted

## Q1B 2 Train a SVM for each of the labels, using Gaussian kernels and one versusall classiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation. You are welcome to try to solve the problem with both standardized 1 and raw attributes and report the results.



In [23]:
y_pred = {}
avg_hamming_gaussian=[]
for i in ['Family','Genus','Species']:
    parameters = [{'kernel': ['rbf'],
               'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5],
                'C': [1, 10, 100, 1000]}]
    clf = GridSearchCV(SVC(kernel='rbf',decision_function_shape='ovr'), parameters,cv=10)
    clf.fit(X_train, y_train[[i]].values.ravel())
    print("Best parameters set found on development set for class {} :".format(i))
    print()
    print(clf.best_params_)
    y_pred[i]= clf.predict(X_test)
    print(classification_report(y_test[[i]].values.ravel(), y_pred[i]))
    print("The hamming loss for class {} is {}".format(i,hamming_loss(y_test[[i]], y_pred[i])))
    avg_hamming_gaussian.append(1- hamming_loss(y_test[[i]], y_pred[i]))
    
df_match=pd.DataFrame()
for i in ['Family','Genus','Species']:
    df_match[i] = np.where(y_test[i] == y_pred[i],1,0)
df_match['sum'] = df_match[['Family','Genus','Species']].sum(axis=1)
em = np.where(df_match['sum']==3,1.0,0).sum(axis=0)/len(df_match)
print("the exact match is: {}".format(em))
print("Average Hamming Score is {}".format(statistics.mean(avg_hamming_gaussian)))

Best parameters set found on development set for class Family :

{'C': 100, 'gamma': 0.5, 'kernel': 'rbf'}
                 precision    recall  f1-score   support

      Bufonidae       0.93      0.96      0.95        27
  Dendrobatidae       0.99      1.00      1.00       162
        Hylidae       0.99      0.99      0.99       623
Leptodactylidae       1.00      0.99      0.99      1347

      micro avg       0.99      0.99      0.99      2159
      macro avg       0.98      0.99      0.98      2159
   weighted avg       0.99      0.99      0.99      2159

The hamming loss for class Family is 0.007410838351088467
Best parameters set found on development set for class Genus :

{'C': 10, 'gamma': 0.5, 'kernel': 'rbf'}
               precision    recall  f1-score   support

    Adenomera       1.00      0.99      0.99      1251
     Ameerega       0.99      1.00      1.00       162
Dendropsophus       0.96      0.94      0.95        84
    Hypsiboas       0.97      1.00      0.99      

## Q1 B 3Repeat 1(b)ii and L1-penalized SVMs.2 Remember to standardize3 the at-tributes

In [24]:
#Scaling/standardizing the data
data_y=data[['Family','Genus','Species']]
data_x=data.drop(['Family','Genus','Species','RecordID'], axis=1)
#data_y = preprocessing.scale(data_y)
data_x = preprocessing.scale(data_x)
X_train, X_test, y_train, y_test = train_test_split(data_x,data_y, test_size=0.30, random_state=42)

In [25]:
y_pred = {}
avg_hamming_L1=[]
from sklearn.svm import LinearSVC
for i in ['Family','Genus','Species']:
    parameters = [{'C': [1, 10, 100,1000],'max_iter':[2000]}]
    clf = GridSearchCV(LinearSVC(multi_class='ovr',penalty='l1',loss='squared_hinge',dual=False), parameters,cv=10)
    clf.fit(X_train, y_train[[i]].values.ravel())
    print("Best parameters set found on development set for class {} :".format(i))
    print()
    print(clf.best_params_)
    y_pred[i] = clf.predict(X_test)
    print(classification_report(y_test[[i]].values.ravel(), y_pred[i]))
    print("The hamming loss for class {} is{}".format(i,hamming_loss(y_test[[i]], y_pred[i])))
    avg_hamming_L1.append(1- hamming_loss(y_test[[i]], y_pred[i]))
df_match=pd.DataFrame()
for i in ['Family','Genus','Species']:
    df_match[i] = np.where(y_test[i] == y_pred[i],1,0)
df_match['sum'] = df_match[['Family','Genus','Species']].sum(axis=1)
em = np.where(df_match['sum']==3,1.0,0).sum(axis=0)/len(df_match)
print("the exact match is: {}".format(em))    
print("Average Hamming Score is {}".format(statistics.mean(avg_hamming_L1)))
    

Best parameters set found on development set for class Family :

{'C': 100, 'max_iter': 2000}
                 precision    recall  f1-score   support

      Bufonidae       0.00      0.00      0.00        27
  Dendrobatidae       0.86      0.90      0.88       162
        Hylidae       0.89      0.90      0.90       623
Leptodactylidae       0.96      0.96      0.96      1347

      micro avg       0.93      0.93      0.93      2159
      macro avg       0.68      0.69      0.68      2159
   weighted avg       0.92      0.93      0.92      2159

The hamming loss for class Family is0.07179249652616952
Best parameters set found on development set for class Genus :

{'C': 10, 'max_iter': 2000}
               precision    recall  f1-score   support

    Adenomera       0.97      0.98      0.97      1251
     Ameerega       0.90      0.93      0.92       162
Dendropsophus       0.87      0.71      0.78        84
    Hypsiboas       0.91      0.97      0.94       468
Leptodactylus       0.9

## Q1 b 4Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance. Report your conclusions about the classiers you trained.

In [26]:
y_pred={}
avg_hamming_smote=[]
from imblearn.over_sampling import SMOTE
from sklearn.svm import LinearSVC
for i in ['Family','Genus','Species']:
    smote = SMOTE(random_state=0,sampling_strategy='minority')
    X_smote = X_train
    y_smote = y_train 
    a = smote.fit_sample(X_smote,y_train[i])
    X_smote1 = pd.DataFrame(a[0])
    Y_smote1 = pd.DataFrame(a[1],columns=[i])
   
  
    parameters = [{'C': [1, 10, 100,1000],'max_iter':[2000]}]
    clf = GridSearchCV(LinearSVC(multi_class='ovr',penalty='l1',loss='squared_hinge',dual=False), parameters,cv=10)
    clf.fit(X_smote1, Y_smote1.values.ravel())
    print("Best parameters set found on development set for class {} :".format(i))
    print()
    print(clf.best_params_)
    y_pred[i] = clf.predict(X_test)
    print(classification_report(y_test[[i]].values.ravel(), y_pred[i]))
    print("The hamming distance loss class {} is {}".format(i,hamming_loss(y_test[[i]], y_pred[i])))
    avg_hamming_smote.append(1- hamming_loss(y_test[[i]], y_pred[i]))
df_match=pd.DataFrame()
for i in ['Family','Genus','Species']:
    df_match[i] = np.where(y_test[i] == y_pred[i],1,0)
df_match['sum'] = df_match[['Family','Genus','Species']].sum(axis=1)
em = np.where(df_match['sum']==3,1.0,0).sum(axis=0)/len(df_match)
print("the exact match is: {}".format(em))
print("Average Hamming Score is {}".format(statistics.mean(avg_hamming_smote)))

Best parameters set found on development set for class Family :

{'C': 100, 'max_iter': 2000}
                 precision    recall  f1-score   support

      Bufonidae       0.33      0.93      0.49        27
  Dendrobatidae       0.85      0.89      0.87       162
        Hylidae       0.93      0.88      0.90       623
Leptodactylidae       0.96      0.94      0.95      1347

      micro avg       0.92      0.92      0.92      2159
      macro avg       0.77      0.91      0.80      2159
   weighted avg       0.94      0.92      0.93      2159

The hamming distance loss class Family is 0.07874015748031496
Best parameters set found on development set for class Genus :

{'C': 10, 'max_iter': 2000}
               precision    recall  f1-score   support

    Adenomera       0.97      0.97      0.97      1251
     Ameerega       0.90      0.91      0.90       162
Dendropsophus       0.87      0.63      0.73        84
    Hypsiboas       0.94      0.96      0.95       468
Leptodactylus    

#### Conclusion : As per the above observation of the excat match score as well as hamming score the gaussian kernal SVM is the best classifier as it is having the max exact match score and maximum hamming score

## Q2 K-Means Clustering on a Multi-Class and Multi-Label Data SetMonte-Carlo Simulation: Perform the following procedures 50 times, and report the average and standard deviation of the 50 Hamming Distances that you calculate. (a) Use k-means clustering on the whole Anuran Calls (MFCCs) Data Set (do not split the data into train and test, as we are not performing supervised learning in this exercise). Choose k automatically based on one of the methods provided in the slides (CH or Gap Statistics or scree plots or Silhouettes) or any other method you know. (b) In each cluster, determine which family is the majority by reading the true labels. Repeat for genus and species. (c) Now for each cluster you have a majority label triplet (family, genus, species). Calculate the average Hamming distance (score) between the true labels and the labels assigned by clusters.

In [29]:
# 2nd part 
# Answers of the Second question is printed at the bottom.
import pandas as pd 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import hamming_loss
import random as rand
import statistics
from collections import defaultdict
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
hamming_score_50=[]
for y in range(0,51):
    data= pd.read_csv('Frogs_MFCCs.csv')
    data_y=data[['Family','Genus','Species']]
    data_x= data.drop(['Family','Genus','Species','RecordID'], axis=1)

    
    c=[]
    silhouette_avg=defaultdict(list)
    for i in range(2,15):
        kmeans=KMeans(n_clusters=i,init='k-means++',random_state=100)
        kmeans.fit(data_x)
        c.append(kmeans.inertia_)
        cluster_labels = kmeans.fit_predict(data_x)
        #print(cluster_labels)
        silhouette_avg[i].append(silhouette_score(data_x, cluster_labels))
        #sample_silhouette_values = silhouette_samples(x, cluster_labels)

    print("Avg Silhoouette scores{} for{}th run \n".format(silhouette_avg,y))
    print("\n")
    print("\n")  
    print("\n")

    optimal_k = max(silhouette_avg,key=silhouette_avg.get)
    print("optimal value of the clusters is :{} for {}th run \n".format(optimal_k,y))

   #random_state=rand.randint(0,150)
    
    x = data_x
    optkmeans= KMeans(n_clusters=4,init='k-means++',n_init=5)

    cluster1 = optkmeans.fit_predict(x)

    lables=pd.DataFrame(cluster1)
    lables.columns=["Lables"]

    # question 2.b
    datamaj = pd.concat([data_x,data_y,lables],axis=1)
    majoritycalss = {}
    for i in range(0,optimal_k):
        dftemp = datamaj[datamaj.Lables==i]
        majoritycalss[i]=[dftemp['Family'].value_counts().idxmax(),dftemp['Genus'].value_counts().idxmax(),dftemp['Species'].value_counts().idxmax()]
    
    print("Majority triplet for the {}th run is :{} ".format(y,majoritycalss))
    
    df_new = datamaj
    
    df_new['family'] = 'none'
    df_new['genus'] = 'none'
    df_new['species'] = 'none'
    for clas in range(0,optimal_k):
        df_new['family'] = np.where(df_new['Lables']==clas,majoritycalss[clas][0],df_new['family'])
        df_new['genus'] = np.where(df_new['Lables']==clas,majoritycalss[clas][1],df_new['genus'])
        df_new['species'] = np.where(df_new['Lables']==clas,majoritycalss[clas][2],df_new['species'])

    #df_new.sort_values('Lables')

    #df_new_family = df_new['Family']
    #df_new_Genus=df_new['Genus']
    #df_new_species=df_new['Species']

    #datamaj = datamaj.sort_values('Lables')
    #datamaj_family = datamaj['Family']
    #datamaj_genus=datamaj['Genus']
    #datamaj_species=datamaj['Species']
    df_pred = df_new.loc[:,['family','genus','species']].rename(columns={'family':'Family','genus':'Genus','species':'Species'})
    df_true = df_new.loc[:,['Family','Genus','Species']]


    hamming=[]
    for i in ['Family', 'Genus', 'Species']:
        hamming.append(1-hamming_loss(df_new[i], df_pred[i]))
    #hamm.append(hamming_loss(datamaj_genus, df_new_Genus))
    #hamm.append(hamming_loss(datamaj_species, df_new_species))

    hamming_score_50.append(sum(hamming)/3)
    
    #hamm = []
    #print("Hamming Loss for the Family is {}".format(hamming_loss(datamaj_family, df_new_family)))
    #print("Hamming Loss for the Genus is {}".format(hamming_loss(datamaj_genus, df_new_Genus)))
    #print("Hamming Loss for the Species is {}".format(hamming_loss(datamaj_species, df_new_species)))
    #hamm.append(hamming_loss(datamaj_family, df_new_family))
    #hamm.append(hamming_loss(datamaj_genus, df_new_Genus))
    #hamm.append(hamming_loss(datamaj_species, df_new_species))
    #dftemp =(datamaj[datamaj.Lables==3])

    #hamm_score  = map(lambda x:(1-x),hamm)
    #hamm_score=list(hamm_score)

    #hamming_score = sum(hamm_score)/3
    
    #hamming_score_50.append(hamming_score)

print("Standart Deviation of 50 runs {}".format(statistics.stdev(hamming_score_50)))

print("Mean of the 50 runs {}".format(statistics.mean(hamming_score_50)))

Avg Silhoouette scoresdefaultdict(<class 'list'>, {2: [0.3486778410277152], 3: [0.36768245219926315], 4: [0.37867825251598397], 5: [0.37170236413433505], 6: [0.2644052337316085], 7: [0.2606084479447015], 8: [0.25374577027495093], 9: [0.2759610856321847], 10: [0.2745898401150972], 11: [0.273596013498094], 12: [0.2713402111392396], 13: [0.2583446333957717], 14: [0.2845071526887524]}) for0th run 







optimal value of the clusters is :4 for 0th run 

Majority triplet for the 0th run is :{0: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 1: ['Dendrobatidae', 'Ameerega', 'Ameeregatrivittata'], 2: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens'], 3: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae']} 
Avg Silhoouette scoresdefaultdict(<class 'list'>, {2: [0.3486778410277152], 3: [0.36768245219926315], 4: [0.37867825251598397], 5: [0.37170236413433505], 6: [0.2644052337316085], 7: [0.2606084479447015], 8: [0.25374577027495093], 9: [0.2759610856321847], 10: [0.2745898401150972], 11: 

Avg Silhoouette scoresdefaultdict(<class 'list'>, {2: [0.3486778410277152], 3: [0.36768245219926315], 4: [0.37867825251598397], 5: [0.37170236413433505], 6: [0.2644052337316085], 7: [0.2606084479447015], 8: [0.25374577027495093], 9: [0.2759610856321847], 10: [0.2745898401150972], 11: [0.273596013498094], 12: [0.2713402111392396], 13: [0.2583446333957717], 14: [0.2845071526887524]}) for12th run 







optimal value of the clusters is :4 for 12th run 

Majority triplet for the 12th run is :{0: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 1: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens'], 2: ['Dendrobatidae', 'Ameerega', 'Ameeregatrivittata'], 3: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae']} 
Avg Silhoouette scoresdefaultdict(<class 'list'>, {2: [0.3486778410277152], 3: [0.36768245219926315], 4: [0.37867825251598397], 5: [0.37170236413433505], 6: [0.2644052337316085], 7: [0.2606084479447015], 8: [0.25374577027495093], 9: [0.2759610856321847], 10: [0.2745898401150972], 1

Avg Silhoouette scoresdefaultdict(<class 'list'>, {2: [0.3486778410277152], 3: [0.36768245219926315], 4: [0.37867825251598397], 5: [0.37170236413433505], 6: [0.2644052337316085], 7: [0.2606084479447015], 8: [0.25374577027495093], 9: [0.2759610856321847], 10: [0.2745898401150972], 11: [0.273596013498094], 12: [0.2713402111392396], 13: [0.2583446333957717], 14: [0.2845071526887524]}) for24th run 







optimal value of the clusters is :4 for 24th run 

Majority triplet for the 24th run is :{0: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 1: ['Dendrobatidae', 'Ameerega', 'Ameeregatrivittata'], 2: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens'], 3: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae']} 
Avg Silhoouette scoresdefaultdict(<class 'list'>, {2: [0.3486778410277152], 3: [0.36768245219926315], 4: [0.37867825251598397], 5: [0.37170236413433505], 6: [0.2644052337316085], 7: [0.2606084479447015], 8: [0.25374577027495093], 9: [0.2759610856321847], 10: [0.2745898401150972], 1

Avg Silhoouette scoresdefaultdict(<class 'list'>, {2: [0.3486778410277152], 3: [0.36768245219926315], 4: [0.37867825251598397], 5: [0.37170236413433505], 6: [0.2644052337316085], 7: [0.2606084479447015], 8: [0.25374577027495093], 9: [0.2759610856321847], 10: [0.2745898401150972], 11: [0.273596013498094], 12: [0.2713402111392396], 13: [0.2583446333957717], 14: [0.2845071526887524]}) for36th run 







optimal value of the clusters is :4 for 36th run 

Majority triplet for the 36th run is :{0: ['Leptodactylidae', 'Adenomera', 'AdenomeraAndre'], 1: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 2: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae'], 3: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens']} 
Avg Silhoouette scoresdefaultdict(<class 'list'>, {2: [0.3486778410277152], 3: [0.36768245219926315], 4: [0.37867825251598397], 5: [0.37170236413433505], 6: [0.2644052337316085], 7: [0.2606084479447015], 8: [0.25374577027495093], 9: [0.2759610856321847], 10: [0.2745898401150972], 11

Avg Silhoouette scoresdefaultdict(<class 'list'>, {2: [0.3486778410277152], 3: [0.36768245219926315], 4: [0.37867825251598397], 5: [0.37170236413433505], 6: [0.2644052337316085], 7: [0.2606084479447015], 8: [0.25374577027495093], 9: [0.2759610856321847], 10: [0.2745898401150972], 11: [0.273596013498094], 12: [0.2713402111392396], 13: [0.2583446333957717], 14: [0.2845071526887524]}) for48th run 







optimal value of the clusters is :4 for 48th run 

Majority triplet for the 48th run is :{0: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 1: ['Dendrobatidae', 'Ameerega', 'Ameeregatrivittata'], 2: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens'], 3: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae']} 
Avg Silhoouette scoresdefaultdict(<class 'list'>, {2: [0.3486778410277152], 3: [0.36768245219926315], 4: [0.37867825251598397], 5: [0.37170236413433505], 6: [0.2644052337316085], 7: [0.2606084479447015], 8: [0.25374577027495093], 9: [0.2759610856321847], 10: [0.2745898401150972], 1

In [ ]:
# In the above output I have displayed the  silhoouete score for each cluster K and selected the K value with maximum score, calculated the  Majority class for the optimal K value choosen.
# By running the whole procedure for 50 times calculated the mean and std deviation for the 50 runs.